In [11]:
import pandas as pd
import numpy as np
import random
import json
from collections import Counter
import warnings
import numpy as np
import random
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')

df = pd.read_csv('data/data_compiled_final_may_2023.csv')
df['uploader']=[i.split(' ')[0] for i in df['uploader']]
df = df[['ids','upload_date','duration','comment_count','view_count','uploader']]
df = df.fillna(0)
df['upload_date'] = [str(int(i)) for i in df['upload_date']]
df['upload_date'] = pd.to_datetime(df['upload_date'],format='%Y%m%d')
df['sentiment'] = np.random.uniform(low=-1, high=1, size=(len(df),))


In [12]:
'''
uploader
'''
df_uploader = df[['uploader','upload_date','duration','comment_count','view_count','sentiment']]
df_uploader.to_csv('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/uploader.csv',index= False)

'''
dummy data
'''
df_tags = df.copy()
for i in range(150):
    df_tags['tag'+str(i)] = np.random.randint(low = 2,size=len(df_tags))
    df_tags['proba_'+'tag'+str(i)] = np.random.uniform(low=0, high=1, size=(len(df_tags),))

''' 
Faces
'''

with open("data/facerec_results.txt", "r") as fp:
    person_dict = json.load(fp)
df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')

df_faces = df.copy()
faces_id = list(person_dict.keys())
lis_ = []
for f_id in faces_id:
    if f_id in df_attributes.image_id.values:
        faces_dict =  dict(Counter([i[:11] for i in person_dict[f_id]]))
        df_faces[f_id] = [faces_dict[i] if i in faces_dict.keys() else 0 for i in df_faces.ids]

In [13]:
df_faces.to_csv('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/faces.csv', index = False)
df_tags.to_csv('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/tags.csv', index = False)


In [14]:
df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')
df_attributes['Age'] = [int(i) for i in df_attributes['Age'].fillna(np.mean(df_attributes['Age']))]
df_attributes['constituency'] = df_attributes['Name']

poly = Polygon([(17.298641, 79.687922),
 (16.861628, 81.643629),
 (14.884918, 79.717799),
 (14.994391, 77.257505)])

def polygon_random_points (poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_points:
            random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
            if (random_point.within(poly)):
                points.append(random_point)
    return points

points = polygon_random_points(poly,len(df_attributes))
coords = []
for p in points:
    coords.append(str(p.x) +","+str(p.y))
df_attributes['coords'] = coords
df_attributes.to_excel('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/alphapolitica_faces_attributes_coords.xlsx', index = False)
